In [1]:
import sys
import torch
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
from MultilabelEEGNet import MultilabelEEGNet

sys.path.append('../../')
from utils.coco_data_handler import COCODataHandler
from utils.epoch_data_module import EpochDataModule

pl.seed_everything(42, workers=True)

Seed set to 42


42

In [2]:
# data module configuration
subject_session_id = 'subj01_session1'
resample_freq = 256
channel_names = [
    'TP7', 'CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5', 'P7', 'P9', 'PO7', 'PO3', 'O1', 
    'Iz', 'Oz', 'POz', 'Pz', 'CPz', 
    'TP8', 'CP6', 'CP4', 'CP2', 'P2', 'P4', 'P6', 'P8', 'P10', 'PO8', 'PO4', 'O2'
]

data_module = EpochDataModule(channel_names=channel_names, subject_session_id='subj01_session1', resample_freq=resample_freq, shuffle=True, num_workers=4, pin_memory=True)

Opening raw data file s:\PolySecLabProjects\eeg-image-decoding\code\utils\..\..\data\all-joined-1\eeg\preprocessed\ground-truth\subj01_session1_eeg.fif...
    Range : 1121 ... 1777926 =      2.189 ...  3472.512 secs
Ready.
3839 events found on stim channel Status
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179

In [22]:
# model configuration
channel_count = len(channel_names)
time_steps = data_module.data_reader[0][0].shape[1]
num_classes = len(COCODataHandler.get_instance().category_index.keys())

model = MultilabelEEGNet(channel_count, time_steps, num_classes, multi_label_threshold=0.3)

In [23]:
model_checkpoint = ModelCheckpoint(
    monitor='val_loss',
    dirpath='check_points/',
    mode='min',
    filename=f'{subject_session_id}_PO_650ms_{channel_count}_{{epoch:02d}}_{{val_loss:.2f}}',
    auto_insert_metric_name=False,
)

lr_rate_monitor = LearningRateMonitor(logging_interval="epoch")

trainer = pl.Trainer(
    accelerator="auto", # Auto select the best hardware accelerator available
    devices="auto", # Auto select available devices for the accelerator (For eg. mutiple GPUs)
    strategy="auto", # Auto select the distributed training strategy.
    max_epochs=20, # Maximum number of epoch to train for.
    deterministic=True, # For deteministic and reproducible training.
    enable_model_summary=True,
    callbacks=[model_checkpoint, lr_rate_monitor],  # Declaring callbacks to use.
    precision="16", # Using Mixed Precision training.
    logger=True, # Auto generate TensorBoard logs.
)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [24]:
trainer.fit(model, data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                 | Type                 | Params | Mode 
-----------------------------------------------------------------------
0  | ensuredims           | Ensure4d             | 0      | train
1  | dimshuffle           | Expression           | 0      | train
2  | conv_temporal        | Conv2d               | 512    | train
3  | bnorm_temporal       | BatchNorm2d          | 16     | train
4  | conv_spatial         | Conv2dWithConstraint | 464    | train
5  | bnorm_1              | BatchNorm2d          | 32     | train
6  | elu_1                | Expression           | 0      | train
7  | pool_1               | AvgPool2d            | 0      | train
8  | drop_1               | Dropout              | 0      | train
9  | conv_separable_dept  | Conv2d               | 256    | train
10 | conv_separable_point | Conv2d               | 256    | train
11 | bnorm_2              | BatchNorm2d          | 32     | train
12 | elu_2                |

Epoch 19: 100%|██████████| 84/84 [00:31<00:00,  2.64it/s, v_num=1, train_batch_loss=0.444, train_batch_acc=0.783, train_batch_f1=0.0969, val_loss=0.438, val_acc=0.791, val_f1=0.135, train_loss=0.442, train_acc=0.790, train_f1=0.129]

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|██████████| 84/84 [00:31<00:00,  2.64it/s, v_num=1, train_batch_loss=0.444, train_batch_acc=0.783, train_batch_f1=0.0969, val_loss=0.438, val_acc=0.791, val_f1=0.135, train_loss=0.442, train_acc=0.790, train_f1=0.129]


In [25]:
model_checkpoint.best_model_path

'S:\\PolySecLabProjects\\eeg-image-decoding\\code\\lightning\\eegnet\\check_points\\subj01_session1_PO_650ms_29_14_0.43.ckpt'

In [26]:
pretrained = MultilabelEEGNet.load_from_checkpoint(model_checkpoint.best_model_path)

In [27]:
import numpy as np

loader = data_module.test_dataloader()

all_preds = []
all_targets = []

for batch in loader:
    epochs, one_hot_encoding = batch
    
    epochs = epochs.cuda()
    y_pred = pretrained.predict(epochs)

    all_preds.append(y_pred)
    all_targets.append(one_hot_encoding)

all_preds = np.concatenate(all_preds, axis=0)  
all_targets = np.concatenate(all_targets, axis=0)

In [28]:
for i in zip(all_preds, all_targets):
    print(f"Predicted: {i[0]}, Target: {i[1]}")

Predicted: [0 1 0 0 0 0 0 0 0 1 0 1], Target: [1 0 0 0 0 0 0 0 0 1 0 0]
Predicted: [0 0 0 0 0 0 0 0 0 1 0 0], Target: [0 0 0 0 0 0 1 0 0 0 0 0]
Predicted: [0 0 0 0 0 0 0 0 0 1 1 0], Target: [0 0 0 0 0 0 0 0 0 1 1 0]
Predicted: [0 0 0 0 0 0 0 0 0 1 0 0], Target: [0 0 0 0 0 0 0 0 0 1 1 0]
Predicted: [0 1 0 0 0 0 0 0 0 1 0 0], Target: [0 0 0 0 1 0 0 1 0 0 0 0]
Predicted: [0 0 0 0 0 0 0 0 0 1 0 0], Target: [1 0 0 0 0 1 0 0 0 1 0 1]
Predicted: [0 0 0 0 0 0 0 0 0 1 0 0], Target: [0 0 0 0 1 1 1 1 0 0 0 0]
Predicted: [0 0 0 0 0 0 0 0 0 1 0 0], Target: [0 0 0 0 1 1 0 0 0 0 0 0]
Predicted: [0 0 0 0 0 0 0 0 0 1 1 0], Target: [0 0 0 0 1 0 0 0 0 1 0 0]
Predicted: [0 0 0 0 0 1 0 0 0 1 0 0], Target: [0 0 0 0 1 1 0 1 0 1 0 0]
Predicted: [0 0 0 0 0 0 0 0 0 1 0 0], Target: [0 0 0 0 1 1 0 0 0 0 0 0]
Predicted: [0 0 0 0 0 0 0 0 0 1 0 0], Target: [0 0 0 0 0 0 0 0 1 0 0 0]
Predicted: [0 0 0 0 0 0 0 0 0 1 0 0], Target: [0 0 0 0 1 1 0 1 0 1 0 0]
Predicted: [0 0 0 0 0 0 0 0 0 1 0 0], Target: [0 1 0 0 0 0 0 0 0

In [20]:
labels = list(COCODataHandler.get_instance().category_index.keys())

In [29]:
from sklearn.metrics import classification_report
print(classification_report(all_targets, all_preds, target_names=labels))

              precision    recall  f1-score   support

   accessory       0.00      0.00      0.00        26
      animal       0.22      0.13      0.16        47
   appliance       1.00      0.06      0.11        18
  electronic       0.00      0.00      0.00        18
        food       0.00      0.00      0.00        30
   furniture       0.24      0.10      0.14        41
      indoor       0.00      0.00      0.00        22
     kitchen       0.67      0.06      0.11        32
     outdoor       0.00      0.00      0.00        20
      person       0.48      1.00      0.65        93
      sports       0.27      0.09      0.13        34
     vehicle       0.30      0.25      0.27        48

   micro avg       0.41      0.28      0.33       429
   macro avg       0.27      0.14      0.13       429
weighted avg       0.30      0.28      0.23       429
 samples avg       0.42      0.25      0.29       429

